In [ ]:
!python3 --version

In [ ]:
import os
from pathlib import Path

In [ ]:
# Dataset Paths
DATASET_PATH = Path('/mnt/DATA/fharookshaik/major_project/dataset')
TRAIN_IMAGES_DIR_PATH = os.path.join(DATASET_PATH,'Train','images')

TRAIN_CSV_PATH = os.path.join(DATASET_PATH,'Train','train.csv')
VALIDATE_CSV_PATH = os.path.join(DATASET_PATH,'Train','val.csv')

TEST_IMAGES_DIR_PATH = os.path.join(DATASET_PATH,'Test','images')
TEST_CSV_PATH = os.path.join(DATASET_PATH,'Test','test.csv')

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
train_df.head()

In [ ]:
train_df.isnull().sum()

In [ ]:
# Dropping the data where OCR = NaN
train_df.dropna(subset=['OCR'],inplace=True)

In [ ]:
import contractions

In [ ]:
fix_contr = lambda text: contractions.fix(text)

In [ ]:
train_df['OCR'] = train_df['OCR'].apply(fix_contr)
train_df.head()

In [ ]:
train_df.fillna('null',inplace=True)
train_df.head()

In [ ]:
import numpy as np

In [ ]:
def str_to_word_list(text):
    if text != 'null':
        # print(text)
        text = text.strip('][').split(', ')
        return [word.strip("\'") for word in text]
    return None

In [ ]:
entities = []
enty_cols = ['hero','villain','victim','other']
for idx,val in train_df.iterrows():
    for enty in enty_cols:
        # str_to_word_list(val.get(enty))
        words = str_to_word_list(val.get(enty))
        if words is not None:
            for word in words:
                entities.append(word)

len(entities)

In [ ]:
from textblob import TextBlob
import nltk

def get_keyword_sentences(text, keyword):
    text = text.replace('\n',' ')
    sentences = nltk.sent_tokenize(text)

    sent_of_interest = []
    for sent in sentences:
        if keyword in sent.lower():
            sent_of_interest.append(sent)
    return sent_of_interest if len(sent_of_interest) > 0 else False

def get_sentiment(list_of_sent):
    sentiment = 0
    count = 0

    if list_of_sent != False:
        for sent in list_of_sent:
            blob = TextBlob(sent)
            sentiment += blob.sentiment.polarity
            count += len(sent)
    return sentiment/count if count!=0 else 0

def extract_sentiment(df,keywords,top=10):
    """df: data for the cluster, keywords: ner for that cluster"""
    print('Extracting Sentiments using TextBlob...')
    # keywords = sorted(keywords.items(),key=lambda x: x[1],reverse=True)
    sentiment_dict = {}
    for keyword in keywords:
        df['sentences'] = df['OCR'].apply(get_keyword_sentences,keyword=keyword.lower())
        keyword_sentiment = df['sentences'].apply(get_sentiment).sum()
        sentiment_dict[keyword] = keyword_sentiment
    return sentiment_dict

In [ ]:
extract_sentiment(train_df,entities)

In [ ]:
nltk.download('punkt')